In [4]:
# Import Spark Session
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession \
        .builder \
        .appName("PySpark Example") \
        .config("spark.some.config.option", "config-value") \
        .getOrCreate()

25/01/16 16:58:20 WARN Utils: Your hostname, codespaces-c9859e resolves to a loopback address: 127.0.0.1; using 10.0.1.137 instead (on interface eth0)
25/01/16 16:58:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/16 16:58:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [8]:
account = spark.read.csv ('./data/accounts.csv', header=True, inferSchema=True)

In [12]:
account.show()

+---+-----------+---------+--------------------+------+------------+-------------+-----+-------+------------+
| id| first_name|last_name|               email|gender|  ip_address|         city|state|country|       phone|
+---+-----------+---------+--------------------+------+------------+-------------+-----+-------+------------+
|  1|       John|      Doe|john.doe@example.com|  Male| 192.168.1.1|     New York|   NY|    USA|123-456-7890|
|  2|       Jane|    Smith|jane.smith@exampl...|Female| 192.168.1.2|  Los Angeles|   CA|    USA|123-456-7891|
|  3|        Jim|    Brown|jim.brown@example...|  Male| 192.168.1.3|      Chicago|   IL|    USA|123-456-7892|
|  4|      Emily|    Davis|emily.davis@examp...|Female| 192.168.1.4|      Houston|   TX|    USA|123-456-7893|
|  5|    Michael|   Wilson|michael.wilson@ex...|  Male| 192.168.1.5|      Phoenix|   AZ|    USA|123-456-7894|
|  6|      Sarah|  Johnson|sarah.johnson@exa...|Female| 192.168.1.6| Philadelphia|   PA|    USA|123-456-7895|
|  7|     

In [13]:
account.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone: string (nullable = true)



In [14]:
transaction = spark.read.parquet ('./data/transactions.parquet', header=True, inferSchema=True)

In [15]:
transaction.show()

+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+
|           column_0|           column_1|            column_2|           column_3|           column_4|           column_5|            column_6|           column_7|           column_8|           column_9|
+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+
|0.06975166797595889| 0.3596639014335491|  0.9579058879421196| 0.2873336206852799| 0.7510577598575394| 0.0453202643241456|  0.4669911300518941|0.11923585143100679| 0.1746633714382202| 0.1651054599812375|
| 0.1309545537499628| 0.5530916563605244| 0.12964005586725436| 0.5768067113989005| 0.4156262361862002|0.20083930841908182|  0.8027172863625173| 0.8947573249020708| 0.8632956555294274|0

In [16]:
transaction.printSchema()

root
 |-- column_0: double (nullable = true)
 |-- column_1: double (nullable = true)
 |-- column_2: double (nullable = true)
 |-- column_3: double (nullable = true)
 |-- column_4: double (nullable = true)
 |-- column_5: double (nullable = true)
 |-- column_6: double (nullable = true)
 |-- column_7: double (nullable = true)
 |-- column_8: double (nullable = true)
 |-- column_9: double (nullable = true)



In [17]:
transaction.count()

100

In [1]:
import os

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd

In [8]:
# Set up the connection
account = "STMKECE-VQ77853"
user = "IAMKENT"
password = "XXXX"
role = "ACCOUNTADMIN"
warehouse = "TEST_WAREHOUSE"
database = "DEV"
schema = "CDM"
table = "ACCOUNTS"

Read Data from CSV

In [5]:
df = pd.read_csv('./data/accounts.csv')
df.columns = [x.upper() for x in df.columns]
df.head()

,ID,FIRST_NAME,LAST_NAME,EMAIL,GENDER,IP_ADDRESS,CITY,STATE,COUNTRY,PHONE
0,1,John,Doe,john.doe@example.com,Male,192.168.1.1,New York,NY,USA,123-456-7890
1,2,Jane,Smith,jane.smith@example.com,Female,192.168.1.2,Los Angeles,CA,USA,123-456-7891
2,3,Jim,Brown,jim.brown@example.com,Male,192.168.1.3,Chicago,IL,USA,123-456-7892
3,4,Emily,Davis,emily.davis@example.com,Female,192.168.1.4,Houston,TX,USA,123-456-7893
4,5,Michael,Wilson,michael.wilson@example.com,Male,192.168.1.5,Phoenix,AZ,USA,123-456-7894


Write to Snowflake

In [9]:
# Make connection to Snowflake
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role=role
)

# Create a cursor object
cur = conn.cursor()

In [10]:
# Execute a command
cur.execute("SELECT current_version()")
one_row = cur.fetchone()
print(one_row[0])

9.1.0


In [12]:
# Load the data into Snowflake from Pandas
df.to_sql(table, conn, if_exists='replace', index=False)

/tmp/ipykernel_1936/696942106.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table, conn, if_exists='replace', index=False)


DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': not all arguments converted during string formatting